<a href="https://colab.research.google.com/github/cmikke97/Automatic-Malware-Signature-Generation/blob/main/src/DetectionBase/DetectionBase_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train and Evaluate ML detection model**

(code available at https://github.com/cmikke97/Automatic-Malware-Signature-Generation)

# **Needed packages**

In [ ]:
!pip install boto3
!pip install baker
!pip install -U logzero
!pip install lmdb

     |████████████████████████████████| 133kB 15.0MB/s 
     |████████████████████████████████| 7.4MB 25.7MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 153kB 64.5MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# **Set up Drive**

In [ ]:
from google.colab import drive

# set path where to mount drive
drive_path = "/content/drive"

# mount drive
drive.mount(drive_path)

Mounted at /content/drive


# **Set base path**

In [ ]:
import os

# set base path (if using google Drive)
base_path = os.path.join(drive_path, "MyDrive/thesis")

# **Clone git repository**

In [ ]:
!rm -r /content/Automatic-Malware-Signature-Generation

rm: cannot remove '/content/Automatic-Malware-Signature-Generation': No such file or directory


In [ ]:
!git clone https://github.com/cmikke97/Automatic-Malware-Signature-Generation.git

Cloning into 'Automatic-Malware-Signature-Generation'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (456/456), done.
remote: Compressing objects: 100% (433/433), done.
Receiving objects: 100% (456/456), 127.15 KiB | 6.05 MiB/s, done.
remote: Total 456 (delta 143), reused 0 (delta 0), pack-reused 0
Resolving deltas: 100% (143/143), done.


## **Download SOREL20M dataset**

In [ ]:
# set destination dir
dataset_dir = "/content/Dataset"

In [ ]:
# execute downloader
!python Automatic-Malware-Signature-Generation/src/DatasetDownloader/sorel20mDownloader.py sorel20m_download $dataset_dir

Now downloading 09-DEC-2020/processed-data/meta.db from s3 bucket..
100% 3788979200/3788979200 [00:33<00:00, 113085201.69it/s]
1/3 done.
Now downloading 09-DEC-2020/processed-data/ember_features/lock.mdb from s3 bucket..
100% 65664/65664 [00:00<00:00, 181365.99it/s]
2/3 done.
Now downloading 09-DEC-2020/processed-data/ember_features/data.mdb from s3 bucket..
100% 76865335296/76865335296 [15:58<00:00, 80169334.77it/s]
3/3 done.


# **Configuration**

To change configuration change values in local copy of "config.py" located at "/content/Automatic-Malware-Signature-Generation/src/DetectionBase/config.py".

# **Train Network**

In [ ]:
os.chdir("/content/Automatic-Malware-Signature-Generation/src/DetectionBase")

import config

checkpoint_base_dir = config.checkpoint_dir

# for the number of configured runs
for i in range(config.runs):
    checkpoint_dir = os.path.join(checkpoint_base_dir, str(i))
    remove_missing_features = os.path.join(base_path, "Dataset/09-DEC-2020/processed-data/shas_missing_ember_features.json")

    # execute train.py script
    !python train.py train_network --checkpoint_dir $checkpoint_dir --remove_missing_features $remove_missing_features

[I 210404 10:27:39 train:151] ...instantiating network
[I 210404 10:27:46 dataset:151] Opening Dataset at /content/Dataset/09-DEC-2020/processed-data/meta.db in train mode.
[I 210404 10:27:47 dataset:161] 400000 samples loaded.
[I 210404 10:27:47 dataset:209] Trying to load shas to ignore from /content/drive/MyDrive/thesis/Dataset/09-DEC-2020/processed-data/shas_missing_ember_features.json...
[I 210404 10:27:51 dataset:220] Dataset now has 393310 samples.
[I 210404 10:27:52 dataset:151] Opening Dataset at /content/Dataset/09-DEC-2020/processed-data/meta.db in validation mode.
[I 210404 10:28:25 dataset:161] 76923 samples loaded.
[I 210404 10:28:25 dataset:209] Trying to load shas to ignore from /content/drive/MyDrive/thesis/Dataset/09-DEC-2020/processed-data/shas_missing_ember_features.json...
[I 210404 10:28:26 dataset:220] Dataset now has 75118 samples.
[I 210404 10:28:26 train:188] Starting training..
 Epoch: 1/10 49/49 total loss: -1.847 malware loss:  0.440 count loss:-23.222 tags

# **Evaluate Network**

In [ ]:
import json

results_base_dir = config.results_dir

#instantiate results_files dictionary
results_files = {}

# for the number of configured runs
for i in range(config.runs):
    # add file path to results_files dictionary (used for plotting results)
    results_files["run_id_" + str(i)] = os.path.join(results_base_dir, str(i), "results.csv");

    results_dir = os.path.join(results_base_dir, str(i))
    checkpoint_file = os.path.join(config.checkpoint_dir, str(i), "epoch_10.pt")
    remove_missing_features = os.path.join(base_path, "Dataset/09-DEC-2020/processed-data/shas_missing_ember_features.json")

    # execute evaluate.py script
    !python evaluate.py evaluate_network --results_dir $results_dir --checkpoint_file $checkpoint_file --remove_missing_features $remove_missing_features
    
# create and open the results.json file in write mode
with open(os.path.join(results_base_dir, "results.json"), "w") as output_file:
    # save results_files dictionary as a json file
    json.dump(results_files, output_file)

[I 210405 01:15:02 dataset:151] Opening Dataset at /content/Dataset/09-DEC-2020/processed-data/meta.db in test mode.
[I 210405 01:15:44 dataset:161] 123077 samples loaded.
[I 210405 01:15:44 dataset:209] Trying to load shas to ignore from /content/drive/MyDrive/thesis/Dataset/09-DEC-2020/processed-data/shas_missing_ember_features.json...
[I 210405 01:15:44 dataset:220] Dataset now has 120760 samples.
[I 210405 01:15:45 evaluate:127] ...running network evaluation
100% 15/15 [13:37<00:00, 54.51s/it]
...done
[I 210405 01:29:35 dataset:151] Opening Dataset at /content/Dataset/09-DEC-2020/processed-data/meta.db in test mode.
[I 210405 01:30:16 dataset:161] 123077 samples loaded.
[I 210405 01:30:16 dataset:209] Trying to load shas to ignore from /content/drive/MyDrive/thesis/Dataset/09-DEC-2020/processed-data/shas_missing_ember_features.json...
[I 210405 01:30:17 dataset:220] Dataset now has 120760 samples.
[I 210405 01:30:17 evaluate:127] ...running network evaluation
100% 15/15 [13:33<00:0

# **Plot Results**

In [ ]:
# for the number of configured runs
for i in range(config.runs):
    results_file = os.path.join(results_base_dir, str(i), "results.csv")
    output_filename = os.path.join(results_base_dir, str(i), "results.png")

    # execute plot.py to plot per-tag results for the single run
    !python plots.py plot_tag_result --results_file $results_file --output_filename $output_filename


run_to_filename_json = os.path.join(results_base_dir, "results.json")
output_filename = os.path.join(results_base_dir, "results.png")
tag_to_plot = 'malware'

# execute plot.py to plot the model results mean and confidence (at least 2 runs are needed)
!python plots.py plot_roc_distribution_for_tag --run_to_filename_json $run_to_filename_json --output_filename $output_filename --tag_to_plot $tag_to_plot